# Blcok Recognition

## Import Libraries

In [1]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import open3d as o3d
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Read Test Images

In [2]:
img_color = cv2.imread('test_imgs/jenga_tower_color.png')
img_depth = cv2.imread('test_imgs/jenga_tower_depth.png', 0)
print('color shape: ', img_color.shape)
print('depth shape: ', img_depth.shape)

color shape:  (1536, 2048, 3)
depth shape:  (1536, 2048)


## Extract Blocks' Masks By Colors

In [3]:
height, width = img_color.shape[:2] # 이미지의 높이와 너비 불러옴, 가로 [0], 세로[1]

img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV) # cvtColor 함수를 이용하여 hsv 색공간으로 변환

In [4]:
colors = ['green', 'pink', 'yellow', 'blue', 'violet', 'red']

In [5]:
# RED
lower_red1 = np.array([0, 130, 50])
upper_red1 = np.array([15, 255, 255])
lower_red2 = np.array([160,130,50])
upper_red2 = np.array([179,255,255])

# PINK
lower_pink1 = np.array([0, 70, 80])
upper_pink1 = np.array([10, 130, 255])
lower_pink2 = np.array([150,70,80])
upper_pink2 = np.array([179,130,255])

# GREEN
lower_green = (70-20, 50, 50)
upper_green = (70+15, 255, 255)

# YELLOW
lower_yellow = (30-10, 80, 80)
upper_yellow = (30+10, 255, 255)

# BLUE
lower_blue = (100-10, 100, 100)
upper_blue = (100+9, 255, 255)

# VIOLET
lower_violet = (130-20, 50, 30)
upper_violet = (130+20, 255, 255)

In [6]:
blocks_rgb_by_color = []
blocks_mask_by_color = []
for color in colors:
    if color == 'pink' or color =='red':
        for i in (1,2):
            exec(f"lower_color{i} = lower_{color}{i}")
            exec(f"upper_color{i} = upper_{color}{i}")

        mask_color1 = cv2.inRange(img_hsv, lower_color1, upper_color1)
        mask_color2 = cv2.inRange(img_hsv, lower_color2, upper_color2)
        img_mask_color = mask_color1 + mask_color2
        
        # Denoise by Erosion and Dilation
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 
        
        exec(f"img_result_{color} = img_result_color")
    
    else:
        exec(f"lower_color = lower_{color}")
        exec(f"upper_color = upper_{color}")

        img_mask_color = cv2.inRange(img_hsv, lower_color, upper_color) # 범위내의 픽셀들은 흰색, 나머지 검은색

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        erosion_image_color = cv2.erode(img_mask_color, kernel, iterations=2)  #// make erosion image
        img_mask_color = cv2.dilate(erosion_image_color, kernel, iterations=2)  #// make dilation image

        # 바이너리 이미지를 마스크로 사용하여 원본이미지에서 범위값에 해당하는 영상부분을 획득
        img_result_color = cv2.bitwise_and(img_color, img_color, mask = img_mask_color) 

        exec(f"img_result_{color} = img_result_color")
        
    _, src_bin = cv2.threshold(img_mask_color, 0, 255, cv2.THRESH_OTSU)
    each_color_filtered = cv2.bitwise_and(img_color, img_color, mask = src_bin)
    
    if color == 'blue':
        cv2.imshow('src_bin', src_bin)
        # cv2.imwrite('./test_imgs/blue_mask.png', src_bin)
        cv2.imshow(f'{color}_filtered', each_color_filtered)
        # cv2.imwrite('./test_imgs/blue_extract.png', each_color_filtered)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

    blocks_color = []
    blocks_mask = []

    for i in range(1, cnt): # 각각의 객체 정보에 들어가기 위해 반복문. 범위를 1부터 시작한 이유는 배경을 제외
        (x, y, w, h, area) = stats[i]
        cen_x, cen_y = map(int, centroids[i])
        block_mask = (labels==i)*img_mask_color
        block_color = cv2.bitwise_and(img_color, img_color, mask = block_mask)
        
        # 노이즈 제거
        if area < 600:
            continue
        
        if color == 'blue' and i==2:
            cv2.imshow('blk clr', block_color)
            # cv2.imwrite('./test_imgs/blue_block1_color.png', block_color)
            cv2.imshow('blk msk', block_mask)
            # cv2.imwrite('./test_imgs/blue_block1_mask.png', block_mask)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
        
        blocks_color.append(block_color)
        blocks_mask.append(block_mask)
        
        
    exec(f"blocks_rgb_{color} = blocks_color")
    exec(f"blocks_mask_{color} = blocks_mask")
    exec(f"blocks_rgb_by_color.append(blocks_rgb_{color})")
    exec(f"blocks_mask_by_color.append(blocks_mask_{color})")
    

In [7]:
tower_mask = 0
tower_color = 0
for mask, color in zip(blocks_mask_by_color, blocks_rgb_by_color):
    for block_m in mask:
        tower_mask += block_m
    
    for block_c in color:
        tower_color += block_c

In [8]:
cv2.imshow('tower mask', tower_mask)
# cv2.imwrite('./test_imgs/tower_mask.png', tower_mask)
cv2.imshow('tower color', tower_color)
# cv2.imwrite('./test_imgs/tower_color.png', tower_color)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
for c, b, rgb_m in zip(colors, blocks_mask_by_color, blocks_rgb_by_color):
    print(c)
    print(len(b), "Blocks")

green
6 Blocks
pink
6 Blocks
yellow
6 Blocks
blue
6 Blocks
violet
6 Blocks
red
6 Blocks


## Get PointCloud from RGB Image + Depth Image

In [10]:
# temp intrinsic matrix
intrinsic = o3d.camera.PinholeCameraIntrinsic()
intrinsic.intrinsic_matrix = [[971.179, 0, 1025.07],[0, 970.984, 778.291],[0, 0, 1]]

In [67]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=50, origin=[0, 0, 0])
mesh_frame_small = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=0.0001, origin=[0, 0, 0])

In [12]:
def get_pointcloud_from_color_depth(color_image, depth_image, intrinsic):
    o3d_img = o3d.geometry.Image()
    
    if isinstance(color_image, type(o3d_img)):
        pass
    elif isinstance(color_image, np.ndarray):
        color_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        color_image = o3d.geometry.Image(color_image)
        
    if isinstance(depth_image, type(o3d_img)):
        pass
    elif isinstance(depth_image, np.ndarray):
        depth_image = o3d.geometry.Image(depth_image)
        
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image, depth_image)
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, intrinsic)
    
    return pcd

In [13]:
masked_depth = cv2.bitwise_and(img_depth, img_depth, mask = tower_mask)

In [14]:
tower_pcd = get_pointcloud_from_color_depth(color_image=tower_color, depth_image=masked_depth, intrinsic=intrinsic)

In [15]:
print(tower_pcd)
print(np.asarray(tower_pcd.points))
o3d.visualization.draw_geometries([tower_pcd])

PointCloud with 105303 points.
[[ 3.60051560e-05 -1.69810720e-04  7.45098048e-04]
 [ 3.67723658e-05 -1.69810720e-04  7.45098048e-04]
 [ 3.75395756e-05 -1.69810720e-04  7.45098048e-04]
 ...
 [ 3.50973437e-05  1.22609574e-04  4.54901950e-04]
 [ 3.55657454e-05  1.22609574e-04  4.54901950e-04]
 [ 3.60341472e-05  1.22609574e-04  4.54901950e-04]]


In [16]:
blocks_pcd_by_color = []
all_pcd = []
for color, block_mask in zip(colors, blocks_mask_by_color):
    # print(color)
    blocks_pcd = []
    for msk in block_mask:
        masked_block_rgb = cv2.bitwise_and(tower_color, tower_color, mask = msk)
        masked_block_depth = cv2.bitwise_and(img_depth, img_depth, mask = msk)
        
        # Get Each Block's PointCloud
        pcd = get_pointcloud_from_color_depth(color_image=masked_block_rgb, depth_image=masked_block_depth, intrinsic=intrinsic)
        
        # Remove Outlier Points
        pcd, _ = pcd.remove_radius_outlier(512, 0.0001)
        blocks_pcd.append(pcd)
        all_pcd.append(pcd)
        
        # if color=='blue':
        #     o3d.visualization.draw_geometries([pcd])
    
    exec(f"blocks_pcd_{color} = blocks_pcd")
    exec(f"blocks_pcd_by_color.append(blocks_pcd_{color})")

In [17]:
len(blocks_pcd_by_color)

6

## Tower ICP

In [68]:
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(all_pcd)):
    pcd_combined += all_pcd[point_id]

print(pcd_combined)
# print(len(pcd_combined.points))

o3d.visualization.draw_geometries([pcd_combined, mesh_frame_small])

PointCloud with 103361 points.


In [59]:
points = np.asarray(pcd_combined.points)

print("X min/max:", np.min(points[:, 0]), np.max(points[:, 0]))
print("Y min/max:", np.min(points[:, 1]), np.max(points[:, 1]))
print("Z min/max:", np.min(points[:, 2]), np.max(points[:, 2]))

X min/max: -2.4695839440523115e-05 0.00011722465250520398
Y min/max: -0.00011242164872447274 0.00012329381878134236
Z min/max: 0.0004549019504338503 0.0005647059297189116


In [19]:
mesh_tower = o3d.io.read_triangle_mesh("mesh/jenga_tower_side_xy.stl")
print(mesh_tower)

mesh_tower.compute_vertex_normals()
# o3d.visualization.draw_geometries([mesh])

pcd_target = mesh_tower.sample_points_uniformly(number_of_points=len(pcd_combined.points))
print(pcd_target)
o3d.visualization.draw_geometries([pcd_target])

TriangleMesh with 12 points and 4 triangles.
PointCloud with 103361 points.


In [20]:
np.linalg.inv(np.asarray([[0, 0, -1, 0],
            [1, 0, 0, 0],
            [0, -1, 0, 0],
            [0, 0, 0, 1]]))

array([[ 0.,  1.,  0.,  0.],
       [-0., -0., -1., -0.],
       [-1., -0., -0., -0.],
       [ 0.,  0.,  0.,  1.]])

In [21]:
initial_transform = np.asarray([[0, 0, -1, 0],
                                [1, 0, 0, 0],
                                [0, -1, 0, 0],
                                [0, 0, 0, 1]])

In [22]:
def prepare_icp(source, target):
    source_tmp = copy.deepcopy(source)
    target_tmp = copy.deepcopy(target)
    
    # make the point cloud into right position
    source_tmp.transform(initial_transform)
    
    # resize the target pointcloud to make two pointclouds into same scale
    resize = (np.array(target_tmp.points)[:,2].max() - np.array(target_tmp.points)[:,2].min()) / (np.array(source_tmp.points)[:,2].max() - np.array(source_tmp.points)[:,2].min())
    
    # move the source pcd to do icp
    move = np.array(target_tmp.get_center() - source_tmp.get_center()*resize)
    
    # a = o3d.cpu.pybind.utility.Vector3dVector(np.array(target_tmp.points))
    b = o3d.cpu.pybind.utility.Vector3dVector(np.array(source_tmp.points)*resize + move)
    
    # target_tmp.points = a
    source_tmp.points = b
    
    o3d.visualization.draw_geometries([source_tmp, target_tmp, mesh_frame])
    
    return source_tmp, target_tmp, resize, move

In [23]:
source, target, resize, move = prepare_icp(pcd_combined, pcd_target)

In [24]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp, mesh_frame])

In [25]:
threshold = 10
trans_init = np.asarray([[1, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 1, 0],
                         [0, 0, 0, 1]])
draw_registration_result(source, target, trans_init)

In [26]:
print("Initial alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

Initial alignment
RegistrationResult with fitness=5.185902e-01, inlier_rmse=5.217337e+00, and correspondence_set size of 53602
Access transformation to get result.


In [27]:
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=20000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
draw_registration_result(source, target, reg_p2p.transformation)

RegistrationResult with fitness=9.988487e-01, inlier_rmse=1.293631e+00, and correspondence_set size of 103242
Access transformation to get result.
Transformation is:
[[ 5.14617146e-01 -8.57115968e-01 -2.28344079e-02  2.44628838e+01]
 [ 8.57411862e-01  5.14313094e-01  1.80814697e-02 -2.07782625e+00]
 [-3.75388140e-03 -2.88835265e-02  9.99575735e-01  7.50256789e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [28]:
draw_registration_result(source, target, reg_p2p.transformation)

In [29]:
source_temp = copy.deepcopy(source)

source_temp.transform(reg_p2p.transformation)


PointCloud with 103361 points.

In [30]:
(np.array(source_temp.points))[:,2].min()

2.7347551709338522

## Blocks Transformation

In [31]:
trans = reg_p2p.transformation

In [32]:
trans

array([[ 5.14617146e-01, -8.57115968e-01, -2.28344079e-02,
         2.44628838e+01],
       [ 8.57411862e-01,  5.14313094e-01,  1.80814697e-02,
        -2.07782625e+00],
       [-3.75388140e-03, -2.88835265e-02,  9.99575735e-01,
         7.50256789e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [33]:
def transform_blocks(pcd, icp_transform):
    pcd_temp = copy.deepcopy(pcd)
    pcd_temp.transform(initial_transform)
    aa = o3d.cpu.pybind.utility.Vector3dVector(np.array(pcd_temp.points)*resize + move)
    pcd_temp.points = aa
    pcd_temp.transform(icp_transform)
    
    return pcd_temp

In [34]:
# WHAT IS THE TARGET BLOCK?
target_block_color = 'pink'
target_block_label = 2

In [35]:
for col, pcds in zip(colors, blocks_pcd_by_color):
    # if col != target_block_color:
    #     continue
    print(col)
    
    for idx, pcd in enumerate(pcds):
        if idx != target_block_label:
            continue
        
        # print("--------------------------------")
        print(idx)
        pcd_new = transform_blocks(pcd, trans)
        
        # o3d.visualization.draw_geometries([pcd_new, mesh_frame, pcd_new.get_axis_aligned_bounding_box()])
        
        box_extent = pcd_new.get_axis_aligned_bounding_box().get_extent()
        print("BOX EXTENT : ", box_extent)
        
        center_coordinate = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()).mean(axis=0)
        
        # print(np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points()))
        print("BOX CENTER COORDINATE : ", center_coordinate)
        
        # print("BOX MAX X,Y and MEAN Z Coordinate")
        x_max = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,0].max()
        y_max = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,1].max()
        z_mean = np.array(pcd_new.get_axis_aligned_bounding_box().get_box_points())[:,2].mean()
        
        if box_extent[1] > 70:
            print("PULL DIRECTION : X")
            vector = np.array([-1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 25/2
            cen_y = y_max - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x + 100
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif box_extent[0] > 70:
            print("PULL DIRECTION : Y")
            vector = np.array([0, -1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 75/2
            cen_y = y_max - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y + 100
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[0]) < 10 and box_extent [1] < 20:
            print("PUSH DIRECTION : Y or -Y")
            vector = np.array([0, -1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 25/2
            cen_y = y_max - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y + 150
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif abs(center_coordinate[1]) < 10 and box_extent [0] < 20:
            print("PUSH DIRECTION : X or -X")
            vector = np.array([-1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 75/2
            cen_y = y_max - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x + 150
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif box_extent[1] < 20:
            print("PULL DIRECTION : -X")
            vector = np.array([1, 0, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 25/2
            cen_y = y_max - 75/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x - 100
            target_y = cen_y
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        elif box_extent[0] < 20:
            print("PULL DIRECTION : -Y")
            vector = np.array([0, 1, 0])
            print("-----BLOCK CENTER COORDINATE-----")
            cen_x = x_max - 75/2
            cen_y = y_max - 25/2
            cen_z = z_mean
            print("X :", cen_x)
            print("Y :", cen_y)
            print("Z :", cen_z)
            pt1 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[cen_x, cen_y, z_mean], [cen_x - 0.01, cen_y - 0.01, cen_z - 0.01], [cen_x + 0.01, cen_y + 0.01, cen_z + 0.01]]))
            print("-----TARGET COORDINATE-----")
            target_x = cen_x
            target_y = cen_y - 100
            target_z = cen_z
            print("X :", target_x)
            print("Y :", target_y)
            print("Z :", target_z)
            pt2 = o3d.cpu.pybind.utility.Vector3dVector(np.array([[target_x, target_y, target_z], [target_x - 0.01, target_y - 0.01, target_z - 0.01], [target_x + 0.01, target_y + 0.01, target_z + 0.01]]))
            print("----------------------------------")
            
        mesh_COORD = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10, origin=center_coordinate)
        ptc1 = o3d.geometry.PointCloud()
        ptc1.points = pt1
        ptc1.paint_uniform_color([1, 0, 0])
        ptc2 = o3d.geometry.PointCloud()
        ptc2.points = pt2
        ptc2.paint_uniform_color([0, 0, 1])
        line = o3d.geometry.LineSet.create_from_point_cloud_correspondences(ptc1, ptc2, [(0,0)])
        o3d.visualization.draw_geometries([pcd_new, mesh_frame, pcd_new.get_axis_aligned_bounding_box(), ptc1, ptc2, line])

green
2
BOX EXTENT :  [25.71447711 88.13284609 14.85647618]
BOX CENTER COORDINATE :  [25.87341245 -5.75967991 98.20659042]
PULL DIRECTION : X
-----BLOCK CENTER COORDINATE-----
X : 26.230651004832225
Y : 0.806743134612347
Z : 98.2065904239191
-----TARGET COORDINATE-----
X : 126.23065100483223
Y : 0.806743134612347
Z : 98.2065904239191
----------------------------------
pink
2
BOX EXTENT :  [25.32050886  4.93168353 13.26135488]
BOX CENTER COORDINATE :  [ 1.64991162 36.64371814 97.6720909 ]
PUSH DIRECTION : Y or -Y
-----BLOCK CENTER COORDINATE-----
X : 1.8101660541343634
Y : 1.6095599028406795
Z : 97.67209090417344
-----TARGET COORDINATE-----
X : 1.8101660541343634
Y : 151.6095599028407
Z : 97.67209090417344
----------------------------------
yellow
2
BOX EXTENT :  [27.7091765   7.08222952 14.00591294]
BOX CENTER COORDINATE :  [-25.65355366  35.95248931  98.19957971]
PULL DIRECTION : -X
-----BLOCK CENTER COORDINATE-----
X : -24.298965412047394
Y : 1.9936040673207245
Z : 98.19957970646149


In [36]:
o3d.visualization.draw_geometries([source_temp, mesh_frame, source_temp.get_axis_aligned_bounding_box()])

In [37]:
np.array(source_temp.get_axis_aligned_bounding_box().get_box_points())

array([[-43.67638271, -51.67265248,   2.73475517],
       [ 40.38895475, -51.67265248,   2.73475517],
       [-43.67638271,  41.35095977,   2.73475517],
       [-43.67638271, -51.67265248, 182.88555306],
       [ 40.38895475,  41.35095977, 182.88555306],
       [-43.67638271,  41.35095977, 182.88555306],
       [ 40.38895475, -51.67265248, 182.88555306],
       [ 40.38895475,  41.35095977,   2.73475517]])

In [38]:
temp = copy.deepcopy(source_temp)

aaa = o3d.cpu.pybind.utility.Vector3dVector(np.array(source_temp.points) - move)

temp.points = aaa

In [39]:
o3d.visualization.draw_geometries([temp, mesh_frame])

In [40]:
np.array(temp.points)[:,0].max()

-348.3357877015925

In [41]:
np.array(temp.points)[:,1].max()

55.6527686473691

In [42]:
np.array(source_temp.points)[:,1].max()

41.350959772025924

In [43]:
np.array(source_temp.points)[:,1].min()

-51.672652476347444

In [44]:
np.array(source_temp.points)[:,2].min()

2.7347551709338522

In [45]:
np.array(source_temp.points)[:,2].max()

182.88555306201738

In [46]:
c = np.array([target_x, target_y, target_z, 1])

In [47]:
vector

array([0, 1, 0])

In [48]:
c

array([   1.47298361, -123.18800905,  113.15348385,    1.        ])

In [49]:
np.linalg.inv(trans)

array([[ 5.14617146e-01,  8.57411862e-01, -3.75388140e-03,
        -1.08046502e+01],
       [-8.57115968e-01,  5.14313094e-01, -2.88835265e-02,
         2.20578516e+01],
       [-2.28344079e-02,  1.80814697e-02,  9.99575735e-01,
        -1.53772862e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [50]:
np.inner(np.linalg.inv(trans), c)

array([-116.09425258,  -45.83014387,  110.69064898,    1.        ])

In [51]:
c2 = np.inner(np.linalg.inv(trans), c)[:3]

In [52]:
c2

array([-116.09425258,  -45.83014387,  110.69064898])

In [53]:
c3 = c2 - move
c3

array([-504.81899503,  -31.528335  ,   13.69634998])

In [54]:
c4 = c3 / resize
c4

array([-6.61079222e-04, -4.12875256e-05,  1.79358790e-05])

In [55]:
cc = np.append(c4, 1)
cc

array([-6.61079222e-04, -4.12875256e-05,  1.79358790e-05,  1.00000000e+00])

In [56]:
c5 = np.inner(np.linalg.inv(np.asarray([[0, 0, -1, 0],
            [1, 0, 0, 0],
            [0, -1, 0, 0],
            [0, 0, 0, 1]])), cc)[:3]

In [57]:
c5

array([-4.12875256e-05, -1.79358790e-05,  6.61079222e-04])

In [58]:
resize

763628.5910099454